# Pune Flood Sensors

##  Import necessary packages

In [10]:
from iudx.entity.Entity import Entity

import pandas as pd
import numpy as np
import json
from datetime import date, datetime, timedelta

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from scipy.interpolate import griddata
import geojsoncontour

import ipywidgets as widgets
from ipywidgets import Layout

import warnings

## Defining variables and widgets

In [11]:
# ids of each resource group
group_id="datakaveri.org/04a15c9960ffda227e9546f3f46e629e1fe4132b/rs.iudx.org.in/pune-env-flood"

# widgets for interaction
prompt1=widgets.HTML(value="")
prompt2=widgets.HTML(value="")
gif_address = 'https://www.uttf.com.ua/assets/images/loader2.gif'
select_ndays=widgets.IntSlider(
    value=1,
    min=1,
    max=30,
    step=1,
    description='Days: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
select_col=widgets.Dropdown(
    options=['currentLevel','measuredDistance','referenceLevel'],
    value='currentLevel',
    description='Property:',
    disabled=False,
)
mywidgets=[select_ndays,select_col]
ui=widgets.VBox([select_ndays,prompt1,select_col,prompt2])

## Functions to fetch, prepare and visualize data

#### *Fetch data*

In [13]:
# fetch latest data in the past n days for a city and add/modify required columns
def get_data(ndays):
    for widget in mywidgets:
        widget.disabled=True
    prompt1.value=f'<img src="{gif_address}" height=150 width=150> Fetching data'
    global entity,measures,latest_measures,start_time,end_time,city
    city='Pune'
    entity=Entity(entity_id=group_id)
    latest_measures=entity.latest().reset_index(drop=True)
    end_time = latest_measures['observationDateTime'].sort_values(ascending=False).reset_index(drop=True)[0]
    start_time = (end_time - timedelta(days=ndays,hours=6))
    measures = entity.during_search(
        start_time=start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        end_time=end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
    )
    measures['observationDateTime']=measures['observationDateTime'].apply(lambda x:x.tz_localize(None))
    latest_measures['observationDateTime']=latest_measures['observationDateTime'].apply(lambda x:x.tz_localize(None))
    rs_coordinates={}
    rs_label={}
    for res in entity.resources:
        rs_coordinates[res['id']]=res['location']['geometry']['coordinates']
        rs_label[res['id']]=res['name']
    latest_measures['x_co']=latest_measures['id'].apply(lambda id:rs_coordinates[id][0])
    latest_measures['y_co']=latest_measures['id'].apply(lambda id:rs_coordinates[id][1])
    measures['x_co']=measures['id'].apply(lambda id:rs_coordinates[id][0])
    measures['y_co']=measures['id'].apply(lambda id:rs_coordinates[id][1])
    measures['label']=measures['id'].apply(lambda id:rs_label[id])
    latest_measures['label']=measures['id'].apply(lambda id:rs_label[id])
    for widget in mywidgets:
        widget.disabled=False
    prompt1.value=f'Fetched {measures.shape[0]} records from {len(entity.resources)} resources'

#### *Temporal Visualization*

In [15]:
# plot the measures of a proprty over ndays for the resource with the latest recording
def timeSeriesVis1(column_name, ndays):
    global units
    prop_desc=entity._data_descriptor[column_name]
    units=prop_desc["unitText"]
    prompt2.value=f'{prop_desc["description"]}<br> Unit: {units}'
    sensor_id = measures.sort_values(by='observationDateTime',ascending=False).reset_index(drop=True)['id'][0]
    single_resource_data = measures.query(f"id == '{sensor_id}'")
    sensor_coordinates=[]
    for res in entity.resources:
        if res['id']==sensor_id:
            sensor_coordinates=res['location']['geometry']['coordinates']
    fig = px.line(
        single_resource_data, 
        x="observationDateTime", 
        y=column_name
    )
    display(widgets.HTML(f'<center style="font-size:14px">Temporal sensor reading for \n {column_name.upper()} from {start_time.date()} to {end_time.date()} for resource at {sensor_coordinates}<center>'))
    fig.update_layout(
        xaxis_title="Observed Timestamp",
        yaxis_title="Sensor reading for "+column_name.upper()+" ("+units+")",
        font=dict(
            size=12
        )
    )
    fig.update_xaxes(rangeslider_visible=True)
    fig.show()



In [21]:
# plot the measures of a proprty over ndays for all resources
def timeSeriesVis2(col, ndays):
    column_name=col
    fig = px.line(
        measures, 
        x="observationDateTime", 
        y=column_name,
        color='label'
    )
    display(widgets.HTML(f'<center style="font-size:14px">Temporal sensor reading for {col.upper()} from {start_time.date()} to {end_time.date()} of all sensors<center>'))
    fig.update_layout(
        xaxis_title="Observed Timestamp",
        yaxis_title="Sensor reading for "+col.upper()+" ("+units+")",
        font=dict(
            size=12
        )
    )
    fig.update_xaxes(rangeslider_visible=True)
    fig.show()



In [36]:
def timeSeriesVis3(ndays):
    sensor_id = measures.sort_values(by='observationDateTime',ascending=False).reset_index(drop=True)['id'][0]
    single_resource_data = measures.query(f"id == '{sensor_id}'")
    sensor_coordinates=[]
    for res in entity.resources:
        if res['id']==sensor_id:
            sensor_coordinates=res['location']['geometry']['coordinates']

    fig=go.Figure()
    fig.add_trace(go.Scatter(x=single_resource_data['observationDateTime'],
                            y=single_resource_data['measuredDistance'],
                            name='Measured Distance',
                            line=dict(color='firebrick')))
    fig.add_trace(go.Scatter(x=single_resource_data['observationDateTime'],
                            y=single_resource_data['referenceLevel'],
                            name='Reference Level',
                            line=dict(color='royalblue',dash='dot')))
    fig.update_layout(title='Measured distance and Reference level over time',
                     xaxis_title='Timestamp',
                     yaxis_title='Distance (meters)')
    fig.update_xaxes(rangeslider_visible=True)
    fig.show()

#### *Basic Visualization*

In [23]:
# plot a bar chart for the latest measures of a property at all active resources
def simpleVis1(col):
    column_name=col
    display(widgets.HTML(f'<center style="font-size:14px">Latest temporal sensor reading for {col.upper()} of all sensors<center>'))
    fig = px.bar(latest_measures, x='label', y=column_name)
    fig.update_layout(
        xaxis_title="Sensor Id",
        yaxis_title="Sensor reading for "+col.upper()+" ("+units+")",
        font=dict(
            size=12
        )
    )
    fig.show()



In [38]:
def simpleVis2(ndays):
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=latest_measures['referenceLevel'],
                            y=latest_measures['label'],
                            marker=dict(color='royalblue'),
                            mode='markers',
                            name='Reference Level'))
    fig.add_trace(go.Scatter(x=latest_measures['measuredDistance'],
                            y=latest_measures['label'],
                            marker=dict(color='firebrick'),
                            mode='markers',
                            name='Measured Distance'))
    fig.update_layout(title='Measured distance and Reference level at different locations',
                     yaxis_title='Device Name',
                     xaxis_title='Distance (meters)')
    fig.show()

#### *Spatial Visualization*

In [27]:
def spatialVis1(column_name):
    maxval=max(list(filter(None,latest_measures[column_name])))
    minval=min(list(filter(None,latest_measures[column_name])))
    geomap2 = folium.Map([latest_measures['y_co'].mean(), latest_measures['x_co'].mean()], zoom_start=12, tiles="cartodbpositron")
    for res in entity.resources:
        entity_id = res["id"]
        try:
            val=latest_measures[latest_measures['id']==entity_id]['currentLevel'].values[0]
            if val is not None and val>0:
                folium.Circle(
                  [res["location"]["geometry"]["coordinates"][1], res["location"]["geometry"]["coordinates"][0]],
                  radius=2000*(val-minval)/(maxval-minval),
                  popup = f'{column_name}: {str(val)}',
                  color='b',
                  fill_color=('red' if ((val-minval)/(maxval-minval))>0.6 else 'blue'),
                  fill=True,
                  fill_opacity=0.4
                  ).add_to(geomap2)
        except:
            pass
    display(geomap2)

## Interactive Outputs

In [12]:
ui

In [29]:
widgets.interactive_output(get_data,{'ndays':select_ndays})

Output()

In [30]:
widgets.interactive_output(spatialVis1,{'column_name':select_col})

Output()

In [32]:
widgets.interactive_output(timeSeriesVis1,{'column_name':select_col, 'ndays':select_ndays})

Output()

In [34]:
widgets.interactive_output(timeSeriesVis2,{'col':select_col, 'ndays':select_ndays})

Output()

In [35]:
widgets.interactive_output(simpleVis1,{'col':select_col})

Output()

In [37]:
widgets.interactive_output(timeSeriesVis3,{'ndays':select_ndays})

Output()

In [39]:
widgets.interactive_output(simpleVis2,{'ndays':select_ndays})

Output()